In [8]:
import heapq
import datetime

# Check platform.
import platform
if platform.machine() not in ['x86_64', 'aarch64']:
    raise SystemExit("Unsupported platform!")

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Connecting to clickhouse on remote server
import clickhouse_connect

# Import the channel estimator and some utilities for converting
# the DMRS fields in the right format from the SCF FAPI format that the dataset follows.
from aerial.phy5g.algorithms import ChannelEstimator
from aerial.util.fapi import dmrs_fapi_to_bit_array

!pip3 install torch
!pip3 install torchinfo
# !pip install torch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# !pip install torchinfo
from torchinfo import summary

clickhouse_client = clickhouse_connect.get_client(host='localhost')

!pip install scikit-learn
from sklearn.metrics import r2_score  # Import R² score function

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
def moving_average(arr, window_size):
    weights = np.ones(window_size) / window_size
    return np.convolve(arr, weights, mode='valid')

def get_throughput(arr):
    mv_avg = moving_average(arr, 20)
    largest_numbers = heapq.nlargest(1, mv_avg)
    return sum(largest_numbers) / 1
    # print(len(arr))
    # n = 1
    # if len(arr) < n:
    #     return sum(arr) / len(arr)  # If there are less than 10 elements, take the average of all
    # largest_numbers = heapq.nlargest(n, arr)  # Get the n largest numbers
    # return sum(largest_numbers) / n  # Compute the average

In [10]:
TEST_IDs = [
    '582c3021631e4b8890518c458a83b560',
    '8501c8ec60bc4766a35c2e30eaf42b28',
    '81ce1181052e4a868dfe5a90243b4136',
    'c3b0ccf5e133483595578598d741af89',
    '7b6be542d6264ede9482eaac2f0891f5',
    '26d5937541f34de4ad13acc1ea42b9d7',
    'd9d07b91e64e488c87a04936c478f45d',
    '35599c707523406b887334ca6d3801c3',
    'c18c4abb9757456b81a79adad377ea5a',
    'cb8bbf840bd74f51849d62efcaf6fcdc',
    '743fa1e3f6b142fe83a5237528fd5516',
    '8476930f9a894802b477e26e5db5813f',
]

In [11]:
dataset = []
for test_id in TEST_IDs:
    print(f" --- Test ID: {test_id}")
    query = f"""
            SELECT *
            FROM iperf3db
            WHERE test_id = '{test_id}' AND direction = 0
            ORDER BY timestamp
            """
    # print(query)
    results =  clickhouse_client.query_df(query)
    # print(len(results))
    # print(results)

    start_timestamp = results['timestamp'][0] 
    stop_timestamp = results['timestamp'].iloc[-1]
    print(start_timestamp)
    print(stop_timestamp)
    ul_throughputs = results["throughput_mbps"].to_numpy()
    max_ul_throughput = get_throughput(ul_throughputs)
    print(max_ul_throughput)

    query = f"""
    SELECT * FROM MAC_KPIs 
    WHERE TsTaiNs BETWEEN '{start_timestamp}' AND '{stop_timestamp}'
    ORDER BY TsTaiNs DESC
    """
    # print(query)
    kpis = clickhouse_client.query_df(query)
    # print(results)
    dataset.append({
        "y" : max_ul_throughput,
        "xs" : kpis[['phr', 'wb_cqi', 'pusch_snr', 'rsrp']].to_numpy() / np.array([100, 20, 30, -150]),
    })

 --- Test ID: 582c3021631e4b8890518c458a83b560
2025-04-04 16:58:52
2025-04-04 17:05:39
7.897918666462093
 --- Test ID: 8501c8ec60bc4766a35c2e30eaf42b28
2025-04-04 17:16:20
2025-04-04 17:22:47
9.413855433305454
 --- Test ID: 81ce1181052e4a868dfe5a90243b4136
2025-04-04 17:32:04
2025-04-04 17:38:16
17.918505502766358
 --- Test ID: c3b0ccf5e133483595578598d741af89
2025-04-04 18:31:56
2025-04-04 18:37:52
25.588413707872213
 --- Test ID: 7b6be542d6264ede9482eaac2f0891f5
2025-04-04 18:57:05
2025-04-04 19:03:00
50.171339502038116
 --- Test ID: 26d5937541f34de4ad13acc1ea42b9d7
2025-04-04 19:26:17
2025-04-04 19:32:11
54.286321739154886
 --- Test ID: d9d07b91e64e488c87a04936c478f45d
2025-04-05 07:01:01
2025-04-05 07:06:56
45.201479200002666
 --- Test ID: 35599c707523406b887334ca6d3801c3
2025-04-05 07:31:33
2025-04-05 07:37:28
49.984885127225766
 --- Test ID: c18c4abb9757456b81a79adad377ea5a
2025-04-05 07:47:19
2025-04-05 07:53:13
46.433833492457886
 --- Test ID: cb8bbf840bd74f51849d62efcaf6fcdc
2